In [1]:
import dask
from dask.distributed import Client
import dask.dataframe as dd
import tensorflow as tf
import pandas as pd

#change this to your Saturn Dask Cluster URL
SATURN_DASK_CLUSTER_URL = ''

client = Client(SATURN_DASK_CLUSTER_URL)

In [3]:
%%time
#ETL
cols = ['Trip_Pickup_DateTime','Trip_Dropoff_DateTime','Passenger_Count','Trip_Distance','Start_Lon','Start_Lat','End_Lon','End_Lat','Fare_Amt','Tip_Amt','Total_Amt']

df = dd.read_csv('s3://nyc-tlc/trip data/yellow_tripdata_2009-*.csv', usecols=cols, storage_options={'anon': True})
df.head()

# filter wrong columns
query_frags = [
    'Fare_Amt > 0 and Fare_Amt < 500',
    'Passenger_Count > 0 and Passenger_Count < 6',
    'Start_Lon > -75 and Start_Lon < -73',
    'End_Lon > -75 and End_Lon < -73',
    'Start_Lat	 > 40 and Start_Lat < 42',
    'End_Lat > 40 and End_Lat < 42'
]

df = df.query(' and '.join(query_frags))

df['Trip_Pickup_DateTime'] = df['Trip_Pickup_DateTime'].astype('datetime64[ns]')
df['Trip_Dropoff_DateTime'] = df['Trip_Dropoff_DateTime'].astype('datetime64[ns]')
df['Trip_Pickup_DateTime'] = df['Trip_Pickup_DateTime'].apply(lambda x: x.day, meta=('Trip_Dropoff_DateTime', 'int64'))
df['Trip_Dropoff_DateTime'] = df['Trip_Dropoff_DateTime'].apply(lambda x: x.day, meta=('Trip_Dropoff_DateTime', 'int64'))

df = df.compute()

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:41958 remote=tcp://127.0.0.1:42671>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:41978 remote=tcp://127.0.0.1:42671>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:41982 remote=tcp://127.0.0.1:42671>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:41984 remote=tcp://127.0.0.1:42671>


CPU times: user 27.9 s, sys: 32.4 s, total: 1min
Wall time: 3min 50s


In [4]:
not_fare_cols = [col for col in df.columns if col not in ['Fare_Amt']]

def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(8, activation='relu', input_shape=[len(df[not_fare_cols].keys())]),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1)
        
    ])
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model


model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 88        
_________________________________________________________________
dense_1 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 649
Trainable params: 649
Non-trainable params: 0
_________________________________________________________________


In [5]:
%%time
train = df[not_fare_cols].to_numpy()
labels =  df['Fare_Amt'].to_numpy()

EPOCHS = 5
history = model.fit(
  train, labels, batch_size=512, validation_split=0.25, shuffle=True,
  epochs=EPOCHS)

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

Train on 125990835 samples, validate on 41996945 samples
Epoch 1/5
125990835/125990835 [==============================] - 564s 4us/sample - loss: 0.5257 - mae: 0.3383 - mse: 0.5257 - val_loss: 0.8135 - val_mae: 0.5622 - val_mse: 0.8135
Epoch 2/5
125990835/125990835 [==============================] - 554s 4us/sample - loss: 0.4924 - mae: 0.3259 - mse: 0.4924 - val_loss: 0.8323 - val_mae: 0.5852 - val_mse: 0.8323
Epoch 3/5
125990835/125990835 [==============================] - 554s 4us/sample - loss: 0.4864 - mae: 0.3225 - mse: 0.4864 - val_loss: 0.7751 - val_mae: 0.5438 - val_mse: 0.7751
Epoch 4/5
125990835/125990835 [==============================] - 554s 4us/sample - loss: 0.4828 - mae: 0.3208 - mse: 0.4828 - val_loss: 0.7608 - val_mae: 0.5311 - val_mse: 0.7608
Epoch 5/5
125990835/125990835 [==============================] - 554s 4us/sample - loss: 0.4813 - mae: 0.3203 - mse: 0.4813 - val_loss: 0.7457 - val_mae: 0.5046 - val_mse: 0.7457
CPU times: user 1h 21min 50s, sys: 14min 7s, tot

,loss,mae,mse,val_loss,val_mae,val_mse,epoch
0,0.525740,0.338347,0.525741,0.813508,0.562216,0.813509,0
1,0.492372,0.325925,0.492371,0.832268,0.585209,0.832272,1
2,0.486419,0.322531,0.486414,0.775081,0.543843,0.775082,2
3,0.482775,0.320812,0.482772,0.760801,0.531075,0.760802,3
4,0.481329,0.320313,0.481334,0.745706,0.504571,0.745705,4
